In [21]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [23]:
# Load api key
OPENAI_API_KEY=os.getenv("OPENAI_API_KEY")

In [ ]:
! mkdir -p 'data/paul_graham/'
! curl 'https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/paul_graham/paul_graham_essay.txt' -O 'data/paul_graham/paul_graham_essay.txt'

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 75042  100 75042    0     0   142k      0 --:--:-- --:--:-- --:--:--  143k
curl: (6) Could not resolve host: data


In [ ]:
! pip install llama-index-llms-huggingface

In [7]:
from llama_index.core import SimpleDirectoryReader, VectorStoreIndex

In [8]:
# load documents
documents = SimpleDirectoryReader("./data/paul_graham/").load_data()

In [9]:
import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

from llama_index.llms.huggingface import HuggingFaceLLM
from llama_index.core import Settings

In [ ]:
# Taken from hf https://huggingface.co/Writer/camel-5b-hf
from llama_index.core import PromptTemplate
chat_prompt = PromptTemplate(
    "<|system|>\nYou are a helpful assistant that answers questions based on the provided context.\n<|user|>\nContext: {context_str}\n\nQuestion: {query_str}\n<|assistant|>\n"
)

In [42]:
import torch

In [32]:
llm = HuggingFaceLLM(
    context_window=2048,
    max_new_tokens=256,
    generate_kwargs={"temperature": 0.7, "do_sample": True, "top_k": 50, "top_p": 0.95},  # Updated parameters
    query_wrapper_prompt=chat_prompt,  # Use chat prompt
    tokenizer_name="TinyLlama/TinyLlama-1.1B-Chat-v1.0",
    model_name="TinyLlama/TinyLlama-1.1B-Chat-v1.0",
    device_map="auto",
    tokenizer_kwargs={"max_length": 2048},
    model_kwargs={"torch_dtype": torch.bfloat16}  # Changed to bfloat16 like in docs
)

loading configuration file config.json from cache at /Users/lakshitkhandelwal/.cache/huggingface/hub/models--TinyLlama--TinyLlama-1.1B-Chat-v1.0/snapshots/fe8a4ea1ffedaf415f4da2f062534de366a451e6/config.json
Model config LlamaConfig {
  "architectures": [
    "LlamaForCausalLM"
  ],
  "attention_bias": false,
  "attention_dropout": 0.0,
  "bos_token_id": 1,
  "eos_token_id": 2,
  "head_dim": 64,
  "hidden_act": "silu",
  "hidden_size": 2048,
  "initializer_range": 0.02,
  "intermediate_size": 5632,
  "max_position_embeddings": 2048,
  "mlp_bias": false,
  "model_type": "llama",
  "num_attention_heads": 32,
  "num_hidden_layers": 22,
  "num_key_value_heads": 4,
  "pretraining_tp": 1,
  "rms_norm_eps": 1e-05,
  "rope_scaling": null,
  "rope_theta": 10000.0,
  "tie_word_embeddings": false,
  "torch_dtype": "bfloat16",
  "transformers_version": "4.55.0",
  "use_cache": true,
  "vocab_size": 32000
}

loading weights file model.safetensors from cache at /Users/lakshitkhandelwal/.cache/huggin

Some parameters are on the meta device because they were offloaded to the disk.


loading file tokenizer.model from cache at /Users/lakshitkhandelwal/.cache/huggingface/hub/models--TinyLlama--TinyLlama-1.1B-Chat-v1.0/snapshots/fe8a4ea1ffedaf415f4da2f062534de366a451e6/tokenizer.model
loading file tokenizer.json from cache at /Users/lakshitkhandelwal/.cache/huggingface/hub/models--TinyLlama--TinyLlama-1.1B-Chat-v1.0/snapshots/fe8a4ea1ffedaf415f4da2f062534de366a451e6/tokenizer.json
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at /Users/lakshitkhandelwal/.cache/huggingface/hub/models--TinyLlama--TinyLlama-1.1B-Chat-v1.0/snapshots/fe8a4ea1ffedaf415f4da2f062534de366a451e6/special_tokens_map.json
loading file tokenizer_config.json from cache at /Users/lakshitkhandelwal/.cache/huggingface/hub/models--TinyLlama--TinyLlama-1.1B-Chat-v1.0/snapshots/fe8a4ea1ffedaf415f4da2f062534de366a451e6/tokenizer_config.json
loading file chat_template.jinja from cache at None


In [33]:
Settings.chunk_size = 512
Settings.llm = llm

In [34]:
index = VectorStoreIndex.from_documents(documents)

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


In [39]:
# set Logging to DEBUG for more detailed outputs
query_engine = index.as_query_engine()
response = query_engine.query("What did the author say in the essay?")

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


In [41]:
print(response)

The author talks about how writing essays and publishing them online has changed the landscape for writers, offering a glimpse into the new generation of essays and the ways they can be published. The author also discusses the inspiration behind starting their own website, and their motivations for writing about topics outside their main field of expertise.
